In [4]:
import numpy as np
import pandas as pd
import os

train_video = []
data_dir = '/Users/sahilkirti/mini-project(new)/change_output'
file_list = os.listdir(data_dir)
print(file_list)
for filename in file_list:
    if filename.endswith('.txt'):
        # Construct the filepath and label based on the filename
        filepath = os.path.join(data_dir, filename)
        descriptors = np.loadtxt(filepath, delimiter='\t')
        print(filepath)

['walk-front_4.txt', 'walk-front_5.txt', 'golf-swing-front_2.txt', 'diving-side_descriptors_with_labels.txt', 'walk-front_1.txt', 'golf-swing-side_2.txt', 'golf-swing-front_3.txt', 'golf-swing-front_1.txt', 'walk-front_2.txt', 'golf-swing-side_1.txt', 'run-side_4.txt', 'kicking-side_1.txt', 'run-side_1.txt', 'kicking-front_1.txt', 'kicking-side_2.txt', 'run-side_2.txt', 'kicking-front_2.txt', 'golf-swing-front_descriptors_with_labels.txt', 'diving-side_1.txt', 'diving-side_3.txt', 'diving-side_5.txt', 'golf-swing-back_1.txt', 'golf-swing-back_2.txt', 'golf-swing-back_3.txt', 'walk-front_9.txt']
/Users/sahilkirti/mini-project(new)/change_output/walk-front_4.txt
/Users/sahilkirti/mini-project(new)/change_output/walk-front_5.txt
/Users/sahilkirti/mini-project(new)/change_output/golf-swing-front_2.txt
/Users/sahilkirti/mini-project(new)/change_output/diving-side_descriptors_with_labels.txt
/Users/sahilkirti/mini-project(new)/change_output/walk-front_1.txt
/Users/sahilkirti/mini-project(new

In [5]:
class VideoRepresentation:
    __slots__ = (
        'filepath',
        'descriptors',
        'pca_descriptors',
        'fisher_vector',
        'label',
        'predicted_label'
    )

    def __init__(self, filepath=None, descriptors=None, label=None):
        self.filepath = filepath
        self.descriptors = descriptors
        self.pca_descriptors = None
        self.fisher_vector = None
        self.label = label
        self.predicted_label = None

In [6]:
train_lines = []
train_videos = []
with open(('/Users/sahilkirti/mini-project(new)/train_label/train.txt'), 'r') as train_f:
    train_lines = train_f.readlines()
for l in train_lines:
    filepath, label = l.split() 
    video_representation = VideoRepresentation(filepath, np.loadtxt(filepath), label)
    train_videos.append(video_representation)
print([v.descriptors for v in train_videos])

[array([[2.73405105e-01, 1.98466748e-01, 2.28502899e-01, ...,
        0.00000000e+00, 2.41744766e+02, 0.00000000e+00],
       [2.64654011e-01, 1.77216247e-01, 2.14566290e-01, ...,
        0.00000000e+00, 2.44697311e+02, 0.00000000e+00],
       [2.10716113e-01, 2.00183794e-01, 2.96895087e-01, ...,
        0.00000000e+00, 2.44209503e+02, 0.00000000e+00],
       ...,
       [7.43162213e-03, 1.95084855e-01, 2.02841580e-01, ...,
        9.40322800e+01, 0.00000000e+00, 0.00000000e+00],
       [5.91867417e-03, 1.87280744e-01, 1.94611847e-01, ...,
        8.57474365e+01, 0.00000000e+00, 0.00000000e+00],
       [3.67569108e-03, 1.49745077e-01, 1.55364648e-01, ...,
        8.05170898e+01, 0.00000000e+00, 0.00000000e+00]]), array([[2.67792732e-01, 2.04017743e-01, 1.53931156e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.61033475e-01, 1.97607070e-01, 1.63062990e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.11050287e-01, 1.99992999e-01, 1

In [7]:
all_train_descriptors = np.concatenate([v.descriptors for v in train_videos], axis=0)
print(f'total number of train descriptors: {all_train_descriptors.shape[0]}')
print(f'length of each train descriptor: {all_train_descriptors.shape[1]}')

total number of train descriptors: 978308
length of each train descriptor: 224


In [8]:
import sklearn.decomposition as decomp
from sklearn.mixture import GaussianMixture as GMM
from sklearn.svm import LinearSVC

pca = decomp.PCA(64)
pca = pca.fit(all_train_descriptors)

In [9]:
pca

PCA(n_components=64)

In [10]:
 # transform descriptors of each video
for v in train_videos:
    v.pca_descriptors = pca.transform(v.descriptors)

In [11]:
# concatenate the pca-transformed descriptors, to not transform the whole data one extra time
all_train_descriptors = np.concatenate([v.pca_descriptors for v in train_videos], axis=0)

In [12]:
print(f'shape of pca descriptor: {all_train_descriptors.shape}')

shape of pca descriptor: (978308, 64)


In [13]:
# learn GMM model
gmm = GMM(n_components=3, covariance_type='diag')
gmm.fit(all_train_descriptors)

GaussianMixture(covariance_type='diag', n_components=3)

In [14]:
def fisher_from_descriptors(vectors, gmm):
    T = vectors.shape[0]
    D = vectors.shape[1]

    # posterior probabilities
    g = gmm.predict_proba(vectors)

    D_h = (np.sum(g, axis=0) - gmm.weights_) / np.sqrt(gmm.weights_)
    D_h = D_h / T  # normalize by number of trajectories

    D_mu_list = []
    for k in range(3):
        D_mu_k = np.zeros(D, dtype=np.float32)
        for t in range(T):
            D_mu_k_t = g[t, k] * (vectors[t, :] - gmm.means_[k]) / gmm.covariances_[k]
            D_mu_k = np.add(D_mu_k, D_mu_k_t)

        D_mu_k = D_mu_k / np.sqrt(gmm.weights_[k])
        D_mu_list.append(D_mu_k)
    D_mu = np.array(D_mu_list).flatten() / T  # K x D flattened, normalized by number of trajectories

    D_sigma_list = []
    for k in range(3):
        D_sigma_k = np.zeros(D, dtype=np.float32)
        for t in range(T):
            D_sigma_k_t = np.sqrt(2) * g[t, k] * (
                    (vectors[t, :] - gmm.means_[k]) ** 2 / gmm.covariances_[k] ** 2 - 1
            )
            D_sigma_k = np.add(D_sigma_k, D_sigma_k_t)

        D_sigma_k = D_sigma_k / np.sqrt(gmm.weights_[k])
        D_sigma_list.append(D_sigma_k)
    D_sigma = np.array(D_sigma_list).flatten() / T  # K x D flattened, normalized by number of trajectories

    fisher_vector = np.concatenate([D_h, D_mu, D_sigma])  # (2 * D + 1) * K
    fisher_vector = np.sqrt(np.abs(fisher_vector)) * np.sign(fisher_vector)  # power normalize the entire vector
    print("shape of fisher_vector:",fisher_vector.shape)
    return fisher_vector

In [15]:
# compute fisher vectors for each train video
for v in train_videos:
    v.fisher_vector = fisher_from_descriptors(v.pca_descriptors, gmm)
print('calculated Fisher vectors')

shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
calculated Fisher vectors


In [16]:
# initialize and fit a linear SVM
svm = LinearSVC()
svm.fit(X=[v.fisher_vector for v in train_videos], y=[v.label for v in train_videos])
print('fitted SVM')

fitted SVM


/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [17]:
test_lines = []
test_videos = []
with open(('/Users/sahilkirti/mini-project(new)/train_label/test.txt'), 'r') as test_f:
    test_lines = test_f.readlines()
for l in test_lines:
    filepath, label = l.split()
    video_representation = VideoRepresentation(filepath, np.loadtxt(filepath), label)
    test_videos.append(video_representation)
print([v.descriptors for v in test_videos])

[array([[3.19853753e-01, 9.21730325e-02, 8.65308195e-02, ...,
        0.00000000e+00, 9.35534477e-01, 3.03226888e-01],
       [3.18233490e-01, 7.69193098e-02, 8.93945768e-02, ...,
        0.00000000e+00, 3.27651858e+00, 4.66478765e-02],
       [3.19995195e-01, 7.82428160e-02, 7.94045180e-02, ...,
        0.00000000e+00, 1.97407436e+00, 1.23674884e-01],
       ...,
       [2.75382191e-01, 2.05838785e-01, 1.22092813e-01, ...,
        0.00000000e+00, 5.28939575e+02, 1.05681190e+01],
       [2.80939966e-01, 1.97662443e-01, 1.21693566e-01, ...,
        0.00000000e+00, 3.90179474e+02, 1.19661932e+01],
       [2.84581274e-01, 2.00712413e-01, 1.17416233e-01, ...,
        0.00000000e+00, 2.69946655e+02, 1.19661932e+01]]), array([[0.14573048, 0.29078573, 0.29078573, ..., 0.        , 0.        ,
        0.        ],
       [0.21131635, 0.28497651, 0.19138815, ..., 0.        , 0.        ,
        0.        ],
       [0.20364839, 0.29140913, 0.19184038, ..., 0.        , 0.        ,
        0.      

In [18]:
# reduce dimension of all test descriptors using pca fitted on train data
for v in test_videos:
    v.pca_descriptors = pca.transform(v.descriptors)
print('reduced dimensions of the test data')

reduced dimensions of the test data


In [19]:
# calculate a fisher vector for each test video based on the gmm model fit on the train data
for v in test_videos:
    v.fisher_vector = fisher_from_descriptors(v.pca_descriptors, gmm)
print('calculated Fisher vectors on the test data')

shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
shape of fisher_vector: (387,)
calculated Fisher vectors on the test data


In [23]:
# predict the labels of the test videos
accuracy = svm.score(X=[v.fisher_vector for v in test_videos], y=[v.label for v in test_videos])
print(f'accuracy: {accuracy*100}')
prediction = svm.predict(X=[v.fisher_vector for v in test_videos])
for i, v in enumerate(test_videos):
    v.predicted_label = prediction[i]
print('prediction by video: index, true label, predicted label, path\n')
for i, v in enumerate(test_videos):
    print(f'{i}    gt: {v.label}    pred: {v.predicted_label}   {v.filepath}')

accuracy: 66.66666666666666
prediction by video: index, true label, predicted label, path

0    gt: diving-side    pred: diving-side   /Users/sahilkirti/mini-project(new)/change_output/diving-side_descriptors_with_labels.txt
1    gt: golf-swing-front    pred: golf-swing-back   /Users/sahilkirti/mini-project(new)/change_output/golf-swing-front_descriptors_with_labels.txt
2    gt: walk-front    pred: walk-front   /Users/sahilkirti/mini-project(new)/change_output/walk-front_9.txt


In [21]:
# import os
# import pandas as pd

# # Define input and output directories
# input_dir = "/Users/sahilkirti/mini-project(new)/new"
# output_dir = "/Users/sahilkirti/mini-project(new)/change_output"

# # Create output directory if it doesn't exist
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Get a list of CSV files in the input directory
# csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# # Process each CSV file
# for csv_file in csv_files:
#     # Read the CSV file
#     df = pd.read_csv(os.path.join(input_dir, csv_file))
    
#     # Drop the first row
#     df = df.iloc[1:]
    
#     # Drop the last column
#     df = df.iloc[:, :-1]
    
#     # Remove '.csv' extension from filename and add '.txt'
#     txt_filename = os.path.splitext(csv_file)[0] + '.txt'
    
#     # Save the data to a text file in the output directory
#     df.to_csv(os.path.join(output_dir, txt_filename), sep='\t', index=False)

In [22]:
import pickle
# Save model to file
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)

In [ ]:
# import pickle

# # Load the saved model from file
# with open('svm_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

# # Now you can use the loaded model to make predictions
# # For example, let's predict the class of a new sample
# new_sample = [[5.1, 3.5, 1.4, 0.2]]  # Example new sample
# predicted_class = loaded_model.predict(new_sample)
# print("Predicted class:", predicted_class)